In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# 문서 로드
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")

docs = loader.load()

print(f"로드한 문서의 페이지 수 : {len(docs)}")

로드한 문서의 페이지 수 : 23


In [3]:
# 문서 분할
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

split_documents = text_splitter.split_documents(docs)

print(f"분할된 청크의 수 : {len(split_documents)}")

분할된 청크의 수 : 43


In [4]:
# 임베딩 생성
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [5]:
# VectorStore 생성
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [6]:
# 검색기 생성
retriever = vectorstore.as_retriever()

In [7]:
# 프롬프트 생성
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""
    대학교 강의실에서 해당 문서로 대학생을 가르치는 대학교 교수님입니다.
    주어진 context를 사용하여 question에 답변하세요.
    질문에 대한 정확한 답변만 해주어야 합니다.
    정확한 답을 모른다면 잘 모르겠다고 답변하세요.
    존댓말을 사용하여야 합니다.
    한국어로 대답하세요.
    
    #Context:
    {context}
    
    #Question:
    {question}
    
    #Answer:
""")

In [8]:
# 모델 생성
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [9]:
# 체인 생성
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [11]:
# 체인 실행

question = "빌게이츠의 견해가 무엇인가요?"

response = chain.invoke(question)

print(response)

빌 게이츠는 5년 내에 일상언어로 모든 작업을 처리할 수 있는 AI 에이전트가 보급되며, 컴퓨터를 사용하는 방식이 완전히 바뀔 것으로 예상하고 있습니다. 그는 AI 에이전트가 의료, 교육, 생산성, 엔터테인먼트·쇼핑 영역에서 고가로 제공되던 서비스의 대중화를 주도할 것이라고 전망하고 있습니다.
